### **Resources**

<hr>

[Project_Inspiration](https://www.youtube.com/watch?v=honnJp7-cCU&t=691s)


## **Notes Related to This Project**

<hr>

Create the `Next.js` app with the following command:

```bash
npx create-next-app@latest

```

Then, we set up the project folder structure as follows:

## **Folder Structure**

### **public**

The `public` folder will contain the static files, such as images and fonts.

### **fonts**

The `fonts` folder will contain custom fonts. Inside the `fonts` folder, we've a typescript file `fonts.ts` that exports the custom fonts.

```typescript
import localFont from "next/font/local";

export const satoshi = localFont({
  src: [
    {
      path: "./Satoshi-Light.woff2",
      weight: "300",
      style: "normal",
    },...
  ],
  variable: "--font-satoshi",
});
```

The above code, loads the custom font files i.e. `Satoshi-Light.woff2`, `Satoshi-Regular.woff2`, etc from the local filesystem and generates a optimized `CSS` `@font-face` rule for each font file. The `variable` property allows us to use the font as a CSS variable.

The `@font-face` rules generated by the `next/font/local` package will be automatically injected into the global CSS of the application, making the fonts available for use throughout the app as below:

```css
@font-face {
  font-family: "__satoshi";
  src: url("/_next/static/media/Satoshi-Regular.woff2") format("woff2");
  font-weight: 400;
  font-style: normal;
}
```

The `@font-face` is a CSS rule that allows us to define custom fonts that can be loaded and used in the application. The `font-family` is set to a unique name (in this case, `__satoshi`), which can be used in the stylesheets to apply the font.

As, the users won't have the `fonts` written in the CSS file, so we use the `next/font/local` package to load the fonts. This package optimizes the loading of the fonts by generating the necessary CSS rules and making them available globally in the application.

Also, `CSS` variables are created for each font, allowing us to use them in our stylesheets like this:

```css
.font_satoshi_variable_abc123 {
  --font-satoshi: "__satoshi";
}
```

Also, `fonts` are preloaded to avoid flashing and layout shifts when the fonts are loaded. This is done by adding the `preload` attribute to the `<link>` tag in the `_document.js` file.

```bash
<link rel="preload" href="/_next/static/media/Satoshi-Regular.woff2" as="font" type="font/woff2" crossorigin="anonymous" />
```

Also, `fonts` are hashed and served from `/_next/static/media/` to ensure that the browser caches them correctly and serves the correct version of the font files.

**Build Time for Fonts**

You're **very close**, and your understanding is almost entirely correct. Let's walk through it step by step and **fine-tune** a few important points to make your mental model crystal clear.

> "After the build we will have an optimized build file..."

Yes! At **build time**, Next.js:

- Processes the font files (like `Satoshi-Regular.woff2`)
- Optimizes them (e.g., hashes file names)
- Generates `@font-face` rules
- Outputs them into the final CSS
- Generates `<link rel="preload" ...>` tags

> "...when the user requests it, the HTML gets downloaded and `<link rel="preload" ...>` runs and downloads the fonts..."

Yes! The **browser parses the HTML**, sees the `<link rel="preload" ...>` tag, and starts downloading the font **early**—before it's even needed—**to reduce render delays**.

Example:

```html
<link
  rel="preload"
  href="/_next/static/media/Satoshi-Regular.woff2"
  as="font"
  type="font/woff2"
  crossorigin="anonymous"
/>
```

This ensures **fast and smooth rendering** (no flashing text).

> "Then after download, the `@font-face` will have the link for the downloaded font and it will use it?"

🟡 **Almost right**, with a small clarification:

- The browser sees your CSS, finds this:

  ```css
  @font-face {
    font-family: "__satoshi";
    src: url("/_next/static/media/Satoshi-Regular.woff2") format("woff2");
    font-weight: 400;
    font-style: normal;
  }
  ```

- Now it knows:

  > "If something uses `font-family: __satoshi`, I should use this file at this weight/style."

- Then, it finds elements styled like:

  ```css
  html.font_satoshi_variable_xyz {
    --font-satoshi: "__satoshi";
  }

  body {
    font-family: var(--font-satoshi), sans-serif;
  }
  ```

- So the browser:

  1. Sees that `--font-satoshi` equals `__satoshi`
  2. Knows `@font-face` defines what `__satoshi` points to
  3. Uses the preloaded font file that’s already been downloaded
  4. Renders the text with `Satoshi`

**Timeline Recap:**

1. **Build time**:

   - Font files hashed and copied to `/_next/static/media/...`
   - `@font-face` CSS is generated
   - `link rel="preload"` is added to the HTML

2. **Client request**:

   - HTML is parsed
   - Fonts are downloaded early via `<link rel="preload">`
   - CSS is parsed, `@font-face` tells how to use those fonts
   - CSS variables like `--font-satoshi` are resolved
   - Text renders instantly using the font

**Note**

After the build, there will not be a single `CSS` file rather there will be multiple CSS files generated for each page and component. This is because Next.js uses **CSS Modules** and **Code Splitting** to optimize the loading of styles. Each page or component will have its own CSS file, which will be loaded only when that page or component is rendered.

Also, there will not be a single `HTML` file rather there will be multiple `HTML` files generated for each page. This is because Next.js uses **Dynamic Routing** to generate the HTML for each page based on the route. But depends on the rendering strategy used, the HTML files can be static or dynamic.

### **app**

The `app` folder will contain the main application code, including pages, components, and styles. Inside the `app` folder, we have the following structure:

```bash
app/
├── global.css
├── layout.tsx
├── page.tsx

```

**global.css**

The `global.css` file will contain global styles for the application. Here, we've

```Css

@theme inline {
  --font-karla: var(--font-geist-karla);
  --font-satoshi: var(--font-satoshi);
  --color-background: var(--background);
  --color-foreground: var(--foreground);
}
```

Which sets up the theme variables for fonts and colors. Once the `browser` loads the `global.css`, it will have access to these variables and can use them in the stylesheets. All the `fonts` would have been downloaded and preloaded by the time the `global.css` is loaded, so the browser will not flash or shift the layout when the fonts are applied.

**layout.tsx**

`import { Karla } from "next/font/google";` loads the `Karla` font from Google Fonts during the build time. This means that the font will be available in the application without needing to load it from an external source at runtime and all the `@font-face` rules will be generated and injected into the global CSS.

```typescript
import { Karla } from "next/font/google";

const karla = Karla({
  subsets: ["latin"],
  weight: ["400", "700"],
});

export default function Layout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en" className={karla.className}>
      <body>{children}</body>
    </html>
  );
}
```

So, when we do as below:

```typescript
const karla = Karla({
  subsets: ["latin"],
  weight: ["400", "700"],
  variable: "--font-karla",
});
```

We will get an object as:

```typescript
karla = {
  className: "karla_class_xyz",
  variable: "karla_variable_xyz",
};
```

Then use it in the `html` tag as:

```typescript
<html lang="en" className={karla.className}>
```


## **Notes Related to Working of NextJs**

<hr>


Next Day understand the NextJs build process completely, we will go through the `layout.tsx` and `page.tsx` files.

**Resources**

[Link1](https://www.dhiwise.com/post/mastering-the-nextjs-build-process-for-optimal-performance)

[Link2](https://sudoshreyansh.medium.com/how-next-js-builds-your-app-6db6059ca34b)

[Link3](https://nextjs.org/docs/14/pages/building-your-application/deploying)

[Link4](https://blog.stackademic.com/understanding-the-next-js-production-build-f8d4eb23e469)

## **NextJs Build**

<hr>

When you run the `next build` command, the final build output will include the following:

**To Be Continued...**


## **Understand NextJs Layout and Page (NextJs in Deployment)**

By default, Next.js uses a file-based routing system, where each file in the `app` directory corresponds to a route in the application. The `layout.tsx` file is used to define the layout for the entire application, while the `page.tsx` file is used to define the content for the home page.

**My Question**

### 🧵 Real-World Deployment & Routing Behavior in Next.js – Am I Thinking Right?

I’ve been diving deep into how Next.js works under the hood, especially with the App Router and how layout/page rendering and navigation happen — but I just want to make sure I’ve got it right.

Let’s say we have a route like `/home`, and inside that we have nested routes like `/home/admin`, `/home/login`, etc.

From what I understand:

- The `layout.tsx` inside `app/home/` acts as the shared layout for all its child routes.
- When a user first visits `/home`, the full HTML (including the layout) is rendered on the server and sent to the browser.
- Now if they navigate to `/home/login`, since the layout hasn’t changed, Next.js won’t re-render the whole page — just the part that’s different (`page.tsx` inside `/home/login/`), right?

I’m also trying to understand what exactly happens on the network during that route change.

Is it like this?

- Instead of requesting a whole new HTML file like a traditional multi-page app, the client sends a request to the Next.js server (some API or internal endpoint?).
- That server returns some data (maybe not raw HTML, but something like a component tree or serialized JSX?).
- Then the client JS takes that, figures out what’s new, and updates the DOM inside the already-mounted layout.

Is that how it works in the real world?

Also, when we build the project (`next build`), does it generate separate HTML/CSS/JS chunks per route or component so that only what’s needed is loaded during navigation?

Lastly, I'm used to working with Express where you do something like `http.createServer(...)` — but how does that server part work in Next.js deployments? Is it managed for us (like on Vercel), or do we manually handle server setup?

<hr>

**Answer**

Excellent explanation attempt! You're very close and clearly thinking deeply about **how Next.js works under the hood**, especially with the **App Router**, **server components**, **routing**, and **deployment model**.

Let’s now walk through your scenario and provide **a full, in-depth technical breakdown** — clarifying where you're absolutely right, and where small adjustments are needed.

---

### 🧭 Scenario: You have a path `/home` and sub-paths like `/home/admin`, `/home/login`, etc.

Let’s assume you’re using the **Next.js App Router** with this folder structure:

```
app/
├── layout.tsx               ← Global root layout
├── home/
│   ├── layout.tsx           ← Layout for /home and all subroutes
│   ├── page.tsx             ← Renders /home
│   ├── login/
│   │   └── page.tsx         ← Renders /home/login
│   └── admin/
│       └── page.tsx         ← Renders /home/admin
```

---

### ✅ What Happens at Build Time (`next build`)

1. **Layouts and Pages are compiled**

- Each `layout.tsx` and `page.tsx` is compiled into a **React Server Component** (RSC).
- Next.js uses **Webpack** and **Turbopack** to bundle code per route — only the JS/CSS needed for `/home/login` is included in that route's chunk.

2. **Server code and client code are separated**

- Server components (`layout.tsx`, `page.tsx`) don’t ship to the browser.
- Only **client components** (with `'use client'`) are bundled for the browser.

3. **Font files**, images, and static files are hashed and stored under:

```
.next/static/media/
```

4. **HTML pre-rendering (if using SSG) or bundling for SSR**

- If you use `generateStaticParams` and `export const dynamic = 'force-static'`, Next.js may pre-render `.html` and `.json` files for those routes.
- Otherwise, for SSR (default), the HTML will be generated dynamically at runtime.

---

### 🛰️ Now What Happens at Runtime (after Deployment)

You’re correct that the app doesn’t work like a simple Express server doing `http.createServer()`.

Next.js applications, once deployed, behave as a **hybrid SSR framework + static server + API server**, often hosted on:

- **Node.js server (custom or Vercel’s)**
- **Edge runtimes (like Vercel Edge Functions or Cloudflare Workers)**
- **Docker / standalone deployments**

Let’s go step-by-step through a request.

---

### 📍 User visits `/home` (first time)

1. **Client makes request to `/home`**

   - The server sees this and begins resolving:

     ```
     Root layout.tsx
     └─ home/layout.tsx
        └─ home/page.tsx
     ```

2. **React Server Component Tree is rendered**

   - Server renders HTML starting from outermost layout down to the page.
   - CSS classes, `@font-face`, and metadata are injected.
   - This full HTML is streamed back to the client.

3. **Browser receives and paints HTML**

   - The HTML already contains `div#__next` with all components rendered.
   - Fonts are preloaded.
   - JS bundles are preloaded and begin hydrating.

4. **Hydration happens**

   - Next.js uses React to attach event listeners and state to already-rendered HTML.
   - This allows interactivity (buttons, forms, etc.).

---

### 📍 Then user navigates (via `<Link href="/home/login" />`)

Now comes the part you’re curious about.

### What **doesn't happen**:

> ❌ The browser doesn’t reload the page
> ❌ The browser doesn’t request a full HTML page again
> ❌ It doesn’t download a "plain text HTML file" for insertion

---

### ✅ What actually happens:

1. **The client-side router (`next/navigation`) intercepts the route change.**

   - This is like how React Router works — single-page navigation.

2. **Next.js fetches a JSON payload from the server**

   - This is **not raw HTML**, but **serialized React component output** (React Flight protocol).
   - It looks something like:

     ```json
     {
       "children": [...],
       "props": {...},
       "__type": "react.server.component.tree"
     }
     ```

3. **Client-side React receives this JSON and renders it**

   - React uses this data to rebuild the new component tree (`home/login/page.tsx`).
   - Any layouts that haven’t changed (`home/layout.tsx`, `root layout.tsx`) are **reused**.
   - Only the new part (`home/login/page.tsx`) is rendered and hydrated.

4. **The new DOM gets mounted without full page reload**

   - Browser doesn't refresh.
   - Layout doesn't re-render if it hasn't changed.
   - Only the new route content is added inside the layout.

This is why your `layout.tsx` stays persistent and only inner content changes.

---

## 🔥 Advanced Concepts Involved

| Feature            | How it Works                                                                  |
| ------------------ | ----------------------------------------------------------------------------- |
| **Streaming HTML** | Server can stream layout/page chunks gradually for better TTFB                |
| **React Flight**   | Transports server component output to client as JSON, not HTML                |
| **Server Actions** | You can run functions on the server (like mutations) directly from components |
| **Code Splitting** | Each route has its own CSS/JS chunk                                           |
| **Edge Rendering** | Optional runtime on edge functions for faster performance globally            |

---

## 🛠️ Deployment Options

You asked:

> “How is this deployed? Like `http.createServer()`?”

### ✅ No, you don’t manage `http.createServer()` manually.

Here’s how deployment usually works:

1. **Vercel (default)**

   - Automatically deploys each route as a serverless or edge function.
   - Handles routing, scaling, and caching.

2. **Custom Node Server**

   - You can use `next start` after `next build`.
   - Internally, this sets up:

     ```js
     import http from "http";
     import next from "next";
     const app = next({ dev: false });
     app.prepare().then(() => {
       http
         .createServer((req, res) => {
           app.getRequestHandler()(req, res);
         })
         .listen(3000);
     });
     ```

3. **Docker / Standalone**

   - With `output: 'standalone'` and optional custom server.
   - Bundle everything needed into `.next/standalone`.

---

### ✅ Final Verdict on Your Understanding

| Your Statement                            | Correct? | Clarification                                                                        |
| ----------------------------------------- | -------- | ------------------------------------------------------------------------------------ |
| Layout renders base HTML first?           | ✅       | Yes, it's streamed from the server                                                   |
| On navigation, we don't reload layout?    | ✅       | Correct — it’s persistent                                                            |
| Do we request plain HTML for new pages?   | ❌       | No — we request **React component tree JSON** (React Flight)                         |
| Is there one CSS/HTML file per component? | 🟡       | CSS is split per route, but HTML is rendered dynamically per layout/page             |
| Deployment like Express server?           | ❌       | You don’t write it — but under the hood, Next.js runs a smart, optimized Node server |

---

### 🧪 Want to See It in Action?

Use the browser DevTools → **Network tab** → Filter by "Doc" or "XHR":

- On initial load, you’ll see the full HTML.
- On route changes, you’ll see `.rsc` or `.json` fetches — not full page reloads.

---

<hr>

## **TL;DR**

So, when NextJs builds your app, it optimizes everything — from fonts to layouts to routes. It preloads fonts, generates CSS variables, and ensures that your app is fast and efficient. When users navigate, it uses client-side routing to update only the necessary parts of the page without reloading the entire layout, making the experience smooth and fast.

It divides the components based on routes and based on server or client components, it optimizes the loading of styles and scripts. The deployment is managed by Next.js, which handles the server setup and routing for you, allowing you to focus on building your application without worrying about the underlying infrastructure.

Also, it does not load the entire HTML file for each route change, but rather fetches the necessary data and updates the DOM accordingly, making it a highly efficient framework for building modern web applications.

It calls the API to fetch the required component as `JSON`, which is then used to update the existing layout without a full page reload.

This approach allows for a **fast, responsive user experience** while maintaining the benefits of server-side rendering and static site generation.


## **Routing in Next.js**

<hr>

Next.js uses a file-based routing system, where each file in the `app` directory corresponds to a route in the application. The routing is dynamic and can be nested, allowing for complex applications with multiple levels of navigation.

### **Basic Routing**

In Next.js, the routing is based on the file structure in the `app` directory. Each file represents a route, and the folder structure defines the hierarchy of routes. For example:

```
app/
├── about/
│   └── page.tsx             ← Renders /about
├── blog/
│   ├── [slug]/
│   │   └── page.tsx         ← Renders /blog/[slug]
│   └── page.tsx             ← Renders /blog
├── contact/
│   └── page.tsx             ← Renders /contact
```

### **Dynamic Routing**

Dynamic routing allows you to create routes that can change based on the data or parameters passed to them. In Next.js, you can create dynamic routes by using square brackets in the file name.

For example, if you want to create a route for blog posts that can change based on the post slug, you can create a file named `[slug].tsx` inside the `blog` directory:

```
app/
├── blog/
│   ├── [slug]/
│   │   └── page.tsx         ← Renders /blog/[slug]
│   └── page.tsx             ← Renders /blog
```

### **Nested Routing**

Nested routing allows you to create routes that are organized in a hierarchy. In Next.js, you can create nested routes by creating folders inside the `app` directory.

For example, if you want to create a route for a blog post that has comments, you can create a folder named `comments` inside the `[slug]` folder:

```app/
├── blog/
│   ├── [slug]/
│   │   ├── comments/
│   │   │   └── page.tsx         ← Renders /blog/[slug]/comments
│   │   └── page.tsx             ← Renders /blog/[slug]
│   └── page.tsx                 ← Renders /blog
```

### **Catch-All Routes**

Catch-all routes allow you to create a route that can match multiple segments of the URL. In Next.js, you can create a catch-all route by using three dots (`...`) in the file name.

For example, if you want to create a route that can match any URL under the `blog` directory, you can create a file named `[...slug].tsx` inside the `blog` directory:

```
app/
├── blog/
│   ├── [...slug]/
│   │   └── page.tsx         ← Renders /blog/[...slug]
│   └── page.tsx             ← Renders /blog
```

### **Route Groups**

Route groups allow you to organize routes into logical groups without affecting the URL structure. You can create route groups by wrapping folder names in parentheses.

For example, you can group authentication-related routes together:

```
app/
├── (auth)/
│   ├── login/
│   │   └── page.tsx         ← Renders /login
│   ├── register/
│   │   └── page.tsx         ← Renders /register
│   ├── forgot-password/
│   │   └── page.tsx         ← Renders /forgot-password
│   └── layout.tsx           ← Shared layout for auth pages
├── (dashboard)/
│   ├── analytics/
│   │   └── page.tsx         ← Renders /analytics
│   ├── settings/
│   │   └── page.tsx         ← Renders /settings
│   └── layout.tsx           ← Shared layout for dashboard pages
└── page.tsx                 ← Renders /
```

The `(auth)` and `(dashboard)` folders are route groups that:

- **Don't affect the URL structure** - `/login` not `/(auth)/login`
- **Allow shared layouts** - Each group can have its own `layout.tsx`
- **Help organize code** - Logically group related routes together
- **Enable different styling** - Apply different themes per group

Route groups are particularly useful for:

- **Authentication flows** - Group login, register, reset password pages
- **Admin sections** - Separate admin routes with different layouts
- **Multi-tenant apps** - Different layouts for different user types
- **A/B testing** - Group routes for different experimental layouts
